In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 0: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'
target_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN2_256/'

In [4]:
pretrained =  './log/pggan/128/b0/aug/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 1000
start_epoch = 0
train_batch = 200
test_batch = 200
lr = 0.004
schedule = [50, 250, 500, 750]
momentum = 0.1
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/pggan/128/b0/to_style2/2000shot/self2' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# cutmix
cm_prob = 0.5
cm_prob_init = 0.99
cm_prob_low = 0.01
cm_beta = 1.0

# augmentation
blur_prob = 0.5
blog_sig = 0.5
jpg_prob = 0.5

best_acc = 0

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = '_fc.'

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

In [8]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Dataset

In [9]:
train_dir = os.path.join(source_dir, 'style2/1000_shot_only')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})
student_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.3, 'drop_connect_rate':0.3})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/pggan/128/b0/aug/checkpoint.pth.tar'


In [11]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in teacher_model.parameters())/1000000.0))

    Total params: 4.01M


In [12]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(student_model.parameters())
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=10, after_scheduler=scheduler_cosine)

# Loss

In [13]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Source Loss', 'Train Acc.', 'Valid Acc.', 'Source ACC.', 'Train AUROC', 'Valid AUROC', 'Source AUROC'])

In [14]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_gn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_gn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_gn2): GroupNorm(8, 16, ep

In [15]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [16]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

# Train

In [17]:
def train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    teacher_model.eval()
    student_model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()
    alpha = AverageMeter()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            tt= targets[rand_index]
            boolean = targets==tt
            rand_index = rand_index[boolean]
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[boolean, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]

        # compute output
        with torch.no_grad():
            teacher_outputs = teacher_model(inputs)
            teacher_loss = criterion(teacher_outputs, targets)
            sp_alpha = 0
            sigmoid = nn.Sigmoid()
            sp_alpha += sigmoid(-teacher_loss)
        
        outputs = student_model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
        loss =  loss_main + sp_alpha*loss_sp + sp_alpha*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))
        cls_losses.update(loss_cls, inputs.size(0))
        sp_losses.update(loss_sp, inputs.size(0))
        main_losses.update(loss_main.tolist(), inputs.size(0))
        alpha.update(sp_alpha, inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
#         if batch_idx % 100 == 0:
#             print('{batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
#                      batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    print('Train | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | Alpha:{alp:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, alp=alpha.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [18]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()


    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # compute output
            outputs = model(inputs)
            loss_main = criterion(outputs, targets)
            loss_cls = 0
            loss_sp = 0
            loss_cls = reg_cls(model)
            loss_sp = reg_l2sp(model)
            loss = loss_main + 0*loss_sp + 0*loss_cls

            # measure accuracy and record loss
            prec1 = accuracy(outputs.data, targets.data)
            auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
            losses.update(loss.data.tolist(), inputs.size(0))
            top1.update(prec1[0], inputs.size(0))
            arc.update(auroc, inputs.size(0))
            cls_losses.update(loss_cls, inputs.size(0))
            sp_losses.update(loss_sp, inputs.size(0))
            main_losses.update(loss_main.tolist(), inputs.size(0))


            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('Test | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [19]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc, train_auroc = train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc, test_auroc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
    source_loss, source_acc, source_auroc = test(val_source_loader, student_model, criterion, epoch, use_cuda)


    logger.append([state['lr'], train_loss, test_loss,  source_loss, train_acc, test_acc,source_acc, train_auroc, test_auroc, source_auroc])
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : student_model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()
    
    if (epoch+1) % 100 == 0:
        teacher_model.load_state_dict(student_model.state_dict())


Epoch: [1 | 1000] LR: 0.004000
Train | 20/20 | Loss:1.3000 | MainLoss:1.2558 | Alpha:0.0230 | SPLoss:0.0377 | CLSLoss:1.8819 | top1:51.1579 | AUROC:0.0000
Test | 39/20 | Loss:0.7688 | MainLoss:0.7688 | SPLoss:0.0623 | CLSLoss:1.8606 | top1:52.3846 | AUROC:0.5340
Test | 161/20 | Loss:0.0166 | MainLoss:0.0166 | SPLoss:0.0623 | CLSLoss:1.8606 | top1:99.9502 | AUROC:1.0000

Epoch: [2 | 1000] LR: 0.005200
Train | 20/20 | Loss:0.7750 | MainLoss:0.7307 | Alpha:0.0230 | SPLoss:0.0752 | CLSLoss:1.8508 | top1:52.0263 | AUROC:0.0000
Test | 39/20 | Loss:0.7102 | MainLoss:0.7102 | SPLoss:0.0851 | CLSLoss:1.8416 | top1:53.0513 | AUROC:0.5416
Test | 161/20 | Loss:0.0480 | MainLoss:0.0480 | SPLoss:0.0851 | CLSLoss:1.8416 | top1:99.9502 | AUROC:1.0000

Epoch: [3 | 1000] LR: 0.006400
Train | 20/20 | Loss:0.7492 | MainLoss:0.7049 | Alpha:0.0230 | SPLoss:0.0923 | CLSLoss:1.8337 | top1:52.7895 | AUROC:0.0000
Test | 39/20 | Loss:0.6973 | MainLoss:0.6973 | SPLoss:0.0987 | CLSLoss:1.8253 | top1:53.5385 | AUR

Train | 20/20 | Loss:0.4847 | MainLoss:0.4402 | Alpha:0.0214 | SPLoss:0.6638 | CLSLoss:1.4103 | top1:79.5526 | AUROC:0.0000
Test | 39/20 | Loss:0.4580 | MainLoss:0.4580 | SPLoss:0.7141 | CLSLoss:1.4007 | top1:78.7692 | AUROC:0.8682
Test | 161/20 | Loss:0.0445 | MainLoss:0.0445 | SPLoss:0.7141 | CLSLoss:1.4007 | top1:98.7414 | AUROC:1.0000

Epoch: [24 | 1000] LR: 0.015994
Train | 20/20 | Loss:0.7280 | MainLoss:0.5721 | Alpha:0.0236 | SPLoss:5.2359 | CLSLoss:1.3785 | top1:71.2632 | AUROC:0.0000
Test | 39/20 | Loss:0.5656 | MainLoss:0.5656 | SPLoss:7.6445 | CLSLoss:1.3646 | top1:70.4359 | AUROC:0.7792
Test | 161/20 | Loss:0.1842 | MainLoss:0.1842 | SPLoss:7.6445 | CLSLoss:1.3646 | top1:93.4112 | AUROC:0.9993

Epoch: [25 | 1000] LR: 0.015993
Train | 20/20 | Loss:0.7267 | MainLoss:0.5151 | Alpha:0.0236 | SPLoss:7.6199 | CLSLoss:1.3580 | top1:74.0000 | AUROC:0.0000
Test | 39/20 | Loss:0.5296 | MainLoss:0.5296 | SPLoss:7.5991 | CLSLoss:1.3515 | top1:73.9231 | AUROC:0.8165
Test | 161/20 | Loss

Train | 20/20 | Loss:0.4274 | MainLoss:0.1703 | Alpha:0.0237 | SPLoss:9.7978 | CLSLoss:1.0451 | top1:93.5263 | AUROC:0.0000
Test | 39/20 | Loss:0.2317 | MainLoss:0.2317 | SPLoss:9.7610 | CLSLoss:1.0374 | top1:90.7692 | AUROC:0.9721
Test | 161/20 | Loss:0.8979 | MainLoss:0.8979 | SPLoss:9.7610 | CLSLoss:1.0374 | top1:69.7508 | AUROC:0.9443

Epoch: [46 | 1000] LR: 0.015954
Train | 20/20 | Loss:0.3944 | MainLoss:0.1466 | Alpha:0.0230 | SPLoss:9.7218 | CLSLoss:1.0316 | top1:94.3421 | AUROC:0.0000
Test | 39/20 | Loss:0.2263 | MainLoss:0.2263 | SPLoss:9.6835 | CLSLoss:1.0272 | top1:91.2179 | AUROC:0.9721
Test | 161/20 | Loss:1.0917 | MainLoss:1.0917 | SPLoss:9.6835 | CLSLoss:1.0272 | top1:67.2679 | AUROC:0.9412

Epoch: [47 | 1000] LR: 0.015952
Train | 20/20 | Loss:0.4070 | MainLoss:0.1432 | Alpha:0.0247 | SPLoss:9.6504 | CLSLoss:1.0218 | top1:94.5789 | AUROC:0.0000
Test | 39/20 | Loss:0.2190 | MainLoss:0.2190 | SPLoss:9.6043 | CLSLoss:1.0136 | top1:91.6026 | AUROC:0.9738
Test | 161/20 | Loss

Train | 20/20 | Loss:0.3547 | MainLoss:0.1109 | Alpha:0.0240 | SPLoss:9.2075 | CLSLoss:0.9519 | top1:95.6316 | AUROC:0.0000
Test | 39/20 | Loss:0.2100 | MainLoss:0.2100 | SPLoss:9.2027 | CLSLoss:0.9511 | top1:92.4231 | AUROC:0.9782
Test | 161/20 | Loss:1.2571 | MainLoss:1.2571 | SPLoss:9.2027 | CLSLoss:0.9511 | top1:65.8474 | AUROC:0.9278

Epoch: [68 | 1000] LR: 0.001588
Train | 20/20 | Loss:0.3395 | MainLoss:0.0990 | Alpha:0.0237 | SPLoss:9.1994 | CLSLoss:0.9507 | top1:96.5000 | AUROC:0.0000
Test | 39/20 | Loss:0.2092 | MainLoss:0.2092 | SPLoss:9.1948 | CLSLoss:0.9502 | top1:92.5513 | AUROC:0.9779
Test | 161/20 | Loss:1.2838 | MainLoss:1.2838 | SPLoss:9.1948 | CLSLoss:0.9502 | top1:65.3707 | AUROC:0.9264

Epoch: [69 | 1000] LR: 0.001587
Train | 20/20 | Loss:0.3413 | MainLoss:0.0934 | Alpha:0.0245 | SPLoss:9.1901 | CLSLoss:0.9497 | top1:96.7368 | AUROC:0.0000
Test | 39/20 | Loss:0.2101 | MainLoss:0.2101 | SPLoss:9.1859 | CLSLoss:0.9493 | top1:92.3846 | AUROC:0.9781
Test | 161/20 | Loss

Train | 20/20 | Loss:0.3140 | MainLoss:0.0832 | Alpha:0.0232 | SPLoss:9.0333 | CLSLoss:0.9276 | top1:97.1053 | AUROC:0.0000
Test | 39/20 | Loss:0.2086 | MainLoss:0.2086 | SPLoss:9.0287 | CLSLoss:0.9271 | top1:92.5769 | AUROC:0.9786
Test | 161/20 | Loss:1.2807 | MainLoss:1.2807 | SPLoss:9.0287 | CLSLoss:0.9271 | top1:65.8910 | AUROC:0.9263

Epoch: [90 | 1000] LR: 0.001576
Train | 20/20 | Loss:0.3340 | MainLoss:0.0869 | Alpha:0.0248 | SPLoss:9.0245 | CLSLoss:0.9266 | top1:96.6842 | AUROC:0.0000
Test | 39/20 | Loss:0.2081 | MainLoss:0.2081 | SPLoss:9.0207 | CLSLoss:0.9262 | top1:92.6410 | AUROC:0.9784
Test | 161/20 | Loss:1.2948 | MainLoss:1.2948 | SPLoss:9.0208 | CLSLoss:0.9262 | top1:65.6480 | AUROC:0.9254

Epoch: [91 | 1000] LR: 0.001575
Train | 20/20 | Loss:0.3160 | MainLoss:0.0850 | Alpha:0.0232 | SPLoss:9.0179 | CLSLoss:0.9259 | top1:96.9211 | AUROC:0.0000
Test | 39/20 | Loss:0.2088 | MainLoss:0.2088 | SPLoss:9.0136 | CLSLoss:0.9254 | top1:92.6795 | AUROC:0.9786
Test | 161/20 | Loss